# Build context

During image building, Docker can only access files from the build context. The path to the folder that specifies the build context is only a positional argument of the `docker build` command. 

## Check files

The only option to list files acceptalbe in the context is to build the image based on the docker file specified in the following cell in the context.

In [1]:
cat << EOF > dockerfile
FROM alpine
COPY . /context/
EOF

This docker file simply compiles the full context (`.`) into the container's `context` folder.

In the following example we create folder that we'll use as context and generate some 

In [40]:
mkdir example_context

#!/bin/bash

# Function to create a random string
random_string() {
    cat /dev/urandom | tr -cd 'a-z0-9' | head -c 8
}

# Function to create random files
create_files() {
    local num_files=$1
    local dir=$2
    local i=0
    for ((; i<num_files; i++)); do
        local file_name=$(random_string)
        local file_path="${dir}/${file_name}.txt"
        echo "Creating file: ${file_path}"
        echo "Random content" > "${file_path}"
    done
}

# Function to create random directory tree
create_directory_tree() {
    local current_dir=$1
    local depth=$2
    local max_depth=$3
    local num_dirs=$4
    local num_files=$5

    if [ "$depth" -ge "$max_depth" ]; then
        return
    fi

    mkdir -p "$current_dir"
    create_files "$num_files" "$current_dir"

    local i=0
    for ((; i<num_dirs; i++)); do
        local dir_name=$(random_string)
        local new_dir="${current_dir}/${dir_name}"
        create_directory_tree "$new_dir" $((depth + 1)) "$max_depth" "$num_dirs" "$num_files"
    done
}

# Parameters
MAX_DEPTH=3     # Maximum depth of the directory tree
NUM_DIRS=3      # Number of directories at each level
NUM_FILES=2     # Number of files in each directory
ROOT_DIR="example_context"

# Create the directory tree
create_directory_tree "$ROOT_DIR" 0 "$MAX_DEPTH" "$NUM_DIRS" "$NUM_FILES"

echo "Random file tree generated at ${ROOT_DIR}"


Creating file: example_context/s6b2spw6.txt
Creating file: example_context/5xy5bkpo.txt
Creating file: example_context/hzmr7dtu/pawcsiu9.txt
Creating file: example_context/hzmr7dtu/yt512mdl.txt
Creating file: example_context/hzmr7dtu/t4rg5b1b/v1cr8z5c.txt
Creating file: example_context/hzmr7dtu/t4rg5b1b/a7ieuqmt.txt
Creating file: example_context/hzmr7dtu/bavhnqkd/5ia8fs5a.txt
Creating file: example_context/hzmr7dtu/bavhnqkd/rni06o38.txt
Creating file: example_context/hzmr7dtu/n305x90w/604oj9e0.txt
Creating file: example_context/hzmr7dtu/n305x90w/vk13napo.txt
Creating file: example_context/4jzirht6/b5yfegzy.txt
Creating file: example_context/4jzirht6/f2k5ec7k.txt
Creating file: example_context/4jzirht6/31y1b4vj/ek7f02h6.txt
Creating file: example_context/4jzirht6/31y1b4vj/qmusmhz0.txt
Creating file: example_context/4jzirht6/xlmyblvn/wfx9ra2z.txt
Creating file: example_context/4jzirht6/xlmyblvn/uwrvvuro.txt
Creating file: example_context/4jzirht6/xyyvwddc/ld2r3omh.txt
Creating file: exa

In [39]:
rm -r example_context

In [7]:
docker build -f ./dockerfile -t print_context ./example_context/ &> /dev/null

In [8]:
docker run --rm print_context ls context | tail -n 10

file90
file91
file92
file93
file94
file95
file96
file97
file98
file99


In [9]:
rm -r example_context
rm dockerfile